
# ClockInFace Facial Recognition Attendance API – Interactive Colab Demo

ClockInFace is a production-grade **facial recognition attendance backend** designed for modern workforce management systems.

This Google Colab notebook demonstrates how developers can integrate:
- Face registration
- Facial recognition
- Person management
- Attendance-ready identity workflows

**Official website:** https://clockinface.com  
**API Base URL:** `https://clockinface.com/backend/api/v1/`


## API Credentials

To interact with the ClockInFace API, you need two pieces of information:

1. **Config ID** – the unique identifier of the configuration you want to use.  
2. **Access Token** – a secure token to authenticate your API requests.

> ⚠️ **Important:** You must log in to https://clockinface.com/backend to get these credentials.

**Where to find them:**

1. Go to **Security → Configurations** in the ClockInFace dashboard.
2. Select the configuration you want to use (for example: **Config 1**).
3. Copy the **ID** — this is your `CONFIG_ID` (e.g., `abc123def456ghy789jkl123mno456pq`).
4. Go to **Access Tokens** to generate or view your **Access Token** — this is your `ACCESS_TOKEN`.

Once you have both, replace the placeholders in your code:

In [ ]:
CONFIG_ID = "YOUR_CONFIG_ID"
ACCESS_TOKEN = "YOUR_ACCESS_TOKEN"

## Install dependencies

In [ ]:
!pip install requests

## Helper functions and initial configurations

In [ ]:

import requests
from io import BytesIO

def image_from_url(url):
    r = requests.get(url)
    r.raise_for_status()
    return BytesIO(r.content)

BASE_URL = "https://clockinface.com/backend/api/v1"
HEADERS = {"Authorization": f"Bearer {ACCESS_TOKEN}"}


## Get persons of a configuration

In [ ]:
requests.post(
    f"{BASE_URL}/person/get_persons/",
    headers=HEADERS,
    json={"config_id": CONFIG_ID}
).json()

## Register well-known characters

| ID | Name           | Image |
|----|----------------|-------|
| 1  | Ben Affleck    | <img src="https://cdn.britannica.com/33/242333-050-95A19CE8/Actor-Ben-Affleck-premiere-AIR-March-2023.jpg?w=300" width="100"/> |
| 2  | Tom Cruise     | <img src="https://hips.hearstapps.com/hmg-prod/images/tom-cruise-attends-the-uk-premiere-and-royal-film-news-photo-1688375274.jpg?crop=0.98568xw:1xh;center,top&resize=300:*" width="100"/> |
| 3  | Morgan Freeman | <img src="https://image.tmdb.org/t/p/w500/jPsLqiYGSofU4s6BjrxnefMfabb.jpg" width="100"/> |
| 3  | Morgan Freeman | <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTnhb9AHCLo-jTyKN4_JFew5uZu4wiqRHrS0A&s" width="100"/> |


In [ ]:
import json
import requests

characters = [
    (1, "Ben Affleck", "https://cdn.britannica.com/33/242333-050-95A19CE8/Actor-Ben-Affleck-premiere-AIR-March-2023.jpg?w=300"),
    (2, "Tom Cruise", "https://hips.hearstapps.com/hmg-prod/images/tom-cruise-attends-the-uk-premiere-and-royal-film-news-photo-1688375274.jpg?crop=0.98568xw:1xh;center,top&resize=300:*"),
    (3, "Morgan Freeman", "https://image.tmdb.org/t/p/w500/jPsLqiYGSofU4s6BjrxnefMfabb.jpg"),
    (3, "Morgan Freeman", "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTnhb9AHCLo-jTyKN4_JFew5uZu4wiqRHrS0A&s"),
]

registered = []

for id, name, img_url in characters:
    img_file = image_from_url(img_url)

    payload = {
        "config_id": CONFIG_ID,
        "metadata": json.dumps([
            {"key": "id_card", "label": "ID Card", "value": str(id)},
            {"key": "name", "label": "Name", "value": name}
        ])
    }

    files = {
        "image": (f"{name.replace(' ', '_').lower()}.jpg", img_file, "image/jpeg")
    }

    r = requests.post(
        f"{BASE_URL}/person/register_person/",
        headers=HEADERS,
        data=payload,
        files=files,
    )

    registered.append(r.json())

registered

## Recognize registered persons

In [ ]:
def recognize(img_url):
    payload = {
        "config_id": CONFIG_ID,
    }

    img_file = image_from_url(img_url)

    files = {
        "image": ("unknown.jpg", img_file, "image/jpeg")
    }

    return requests.post(
        f"{BASE_URL}/person/facial_recognition/",
        headers=HEADERS,
        data=payload,
        files=files,
    ).json()

### Recognize registered person
<img src="https://cdn.britannica.com/33/242333-050-95A19CE8/Actor-Ben-Affleck-premiere-AIR-March-2023.jpg?w=300" width="100"/>

In [ ]:
img_url = "https://cdn.britannica.com/33/242333-050-95A19CE8/Actor-Ben-Affleck-premiere-AIR-March-2023.jpg?w=300"
recognize(img_url)

### Recognize registered person
<img src="https://media-cldnry.s-nbcnews.com/image/upload/t_fit-1500w,f_auto,q_auto:best/streams/2012/June/120601/397944-120601-morgan-freeman.jpg" width="100"/>

In [ ]:
img_url = "https://media-cldnry.s-nbcnews.com/image/upload/t_fit-1500w,f_auto,q_auto:best/streams/2012/June/120601/397944-120601-morgan-freeman.jpg"
recognize(img_url)

### Recognize registered person
<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTgw1nYqKNfUx-tbcmeVX_Vh6va7jv4ZlxnFg&s" width="100"/>

In [ ]:
img_url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTgw1nYqKNfUx-tbcmeVX_Vh6va7jv4ZlxnFg&s"
recognize(img_url)

### Recognize an unregistered person
<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTBXF8SSJMrWCa-yzTqBoIBAEIF7i-QCO0jVw&s" width="100"/>

In [ ]:
img_url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTBXF8SSJMrWCa-yzTqBoIBAEIF7i-QCO0jVw&s"
recognize(img_url)

## List all registered persons

In [ ]:
registered = requests.post(
    f"{BASE_URL}/person/get_persons/",
    headers=HEADERS,
    json={"config_id": CONFIG_ID}
).json()

registered

## List all faces of a registered person

In [ ]:
PERSON_UUID = registered['data'][0]['uuid']

requests.post(
    f"{BASE_URL}/person/get_person_faces/",
    headers=HEADERS,
    json={
        "config_id": CONFIG_ID,
        "person_uuid": PERSON_UUID
    }
).json()


## Summary

This notebook shows how to build a **facial recognition attendance system** using the ClockInFace API.

Key features demonstrated:
- Secure token authentication
- Face registration
- Face recognition
- Person and embedding management

Learn more at https://clockinface.com
